In [29]:
# Always reload modules to have the current version
%reload_ext autoreload
%autoreload 2


In [30]:
from ranking.models.fasttext import FastTextModel
from ranking.storage.document_store import DocumentStore
from ranking.util import dataset_paths as dp
from ranking.util import json_lines as jl
from ranking.util.corpus import CorpusReader
import os
import ranking.evaluation.evaluate as ev

corpus_name = 'my-new-corpus.txt'
model_name = 'sg-fastText_model.pkl'

if os.path.exists(model_name):
    model = FastTextModel.load(model_name)
else:
    store = DocumentStore(dp.lemmatized_unique_functions_corpus)
    corpus = CorpusReader(corpus_name)
    model = FastTextModel(store, corpus)
    model.save(model_name)


In [31]:
eval_set = jl.read_jsonl(dp.unified_hoogle_results)
eval_set = eval_set[eval_set['hoogleRes'].str.len() > 0]
fasttext_result = eval_set.apply(
    lambda row: model.rank(row['docQuery'].split(), row['hoogleRes']),
    axis=1,
)
eval_set['fasttextres'] = fasttext_result  
eval_set['fasttextRank'] = eval_set.apply(lambda row: ev.get_rank(row['storageId'], row['fasttextres']), axis=1)

In [32]:
print('Hoogle + FastText:')
ev.evaluate(eval_set['fasttextRank'])


Hoogle + FastText:


,Total,TopOne Total:,TopOne in %,TopTen Total:,TopTen in %,MRR
0,10000,7608,0.7608,8920,0.892,0.814429
